In [1]:
#-----------------------------------------------------------------------
# Librerías y algunas variables
#-----------------------------------------------------------------------
import mne                                                              # pip install mne
mne.set_log_level('WARNING')                                            # Luego averiguar ¿para qué?
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb      
import random                                              # Por ahora lo voy a usar para cambiar los nombres de las columnas

import a_funciones as a_fun  

color = ['green', 'blue','red','cyan', 'magenta', 'yellow','k','w']     # Paleta de colores para diferenciar las ondas
xlabel = 'Muestra'
ylabel = 'Amplitud (uV)'

vector_desfase = [1, 2, 3, 4, 5]
# vector_desfase = [5, 10, 15, 20, 25, 30]
# vector_desfase = [20, 30, 40, 50, 60, 70]
# vector_desfase = [2000, 2000, 2000, 2000, 2000, 2000]
#vector_desfase = [0]                                     # Para graficar la señal sin alteraciones en fase, como referencia.
print(f'vector_desfase: ',vector_desfase)

def DrugSignal(signal, t_flash):
    for j in range(0,7):
        desfase = random.choice(vector_desfase)
        for i in range(0,4200):
            if (t_flash[i,3]==2):
                #signal[t_flash[i,0]-1:t_flash[i,0]+250-1,:] += (erptemplate1*3)    ORIGINAL
                signal[t_flash[i,0]-desfase:t_flash[i,0]+250-desfase,j] += (erptemplate1[:,j])                
    return signal

# Now load the basal EEG stream
# mat = scipy.io.loadmat('./dataset/p300-subject-25.mat')
# mat = scipy.io.loadmat('./dataset/itba/P300S02.mat')

files_path_itba = ['./dataset/itba/P300S01.mat', './dataset/itba/P300S02.mat', 
                    './dataset/itba/P300S03.mat', './dataset/itba/P300S06.mat']
# files_path_itba = ['./dataset/p300-subject-26.mat', './dataset/p300-subject-25.mat', 
#                  './dataset/p300-subject-25.mat', './dataset/p300-subject-26.mat']

#                    './dataset/itba/P300S05.mat', './dataset/itba/P300S06.mat',
#                    './dataset/itba/P300S07.mat', './dataset/itba/P300S08.mat']

repetitions = 48

for ii, path in enumerate(files_path_itba):  
    mat = scipy.io.loadmat('./dataset/ERPTemplate.mat')
    routput = mat['routput']
    # In this ERPTemplate, there are two different template signals that are good.
    erptemplate1 = routput[0][7][0][1][0][0][0][7] 
    # erptemplate2 = routput[0][7][0][1][0][0][0][0] 

    # The original ERPTemplate dataset has a sampling frequency of 256 so I need to perform a small downsampling to 250 Hz
    erptemplate1 = np.delete( erptemplate1, range(0,256,43),0)
    # erptemplate2 = np.delete( erptemplate2, range(0,256,43),0)
    # erptemplate1 = np.load('./dataset/array_DrugAmpERPtemplate.npy')
    # print(erptemplate1)
    # print(erptemplate1.shape)
    
    '''
    #-----------------------------------------------------------------------
    # array_DrugAmpERPtemplate es un vector de 5 templates ERP con 
    # distintas variaciones en amplitud +/- 20% del valor max y min del ERPtemplate original
    #-----------------------------------------------------------------------
    v_min = erptemplate1.min()*20
    v_max = erptemplate1.max()*20
    array_DrugAmpERPtemplate = np.empty(5,dtype=object)
    for i in range(5):
        v_aleatorio = np.random.uniform(v_min, v_max, (1, 8)) 
        #print(f'v_aleatorio({i}): ',v_aleatorio)
        # v_aleatorio = [-5,1,5,0,5,0,5,0] # Vector fijo para testing
        DrugAmpERPtemplate = np.empty_like(erptemplate1) # Inicializo un array igual que erptemplate1
        for j in range(erptemplate1.shape[0]):
            DrugAmpERPtemplate[j, :] = erptemplate1[j, :] + v_aleatorio
        array_DrugAmpERPtemplate[i] = DrugAmpERPtemplate
        #print(f'array_DrugAmpERPtemplate[{i}].min()', array_DrugAmpERPtemplate[i].min())
        #print(f'array_DrugAmpERPtemplate[{i}].max()', array_DrugAmpERPtemplate[i].max())
        #print('----------------------------------------------------')

    # erptemplate1 = array_DrugAmpERPtemplate[1]    
    erptemplate1 = array_DrugAmpERPtemplate[0]
    '''
    results_mne=f'./a_results/20240201_fase.csv' # Si necesitás, con cambiar la extensión a.txt es suficiente
    file_temp=open(results_mne,"a") 
    # print(f'P{ii+1},', file=file_temp) # COLUMNA0.
    punto_mat=scipy.io.loadmat(path)

    # Data point zero for the eight channels.  Should be in V.
    signal = punto_mat['data'][0][0][0] 
    #* pow(10,6)

    # Trials
    t_trials = punto_mat['data'][0][0][3]

    # Flash matrix
    t_flash = punto_mat['data'][0][0][4]

    signal = DrugSignal(signal, t_flash)

    t_stim = punto_mat['data'][0][0][2]
    t_type = punto_mat['data'][0][0][1]

    ch_names=[ 'Fz'  ,  'Cz',    'P3' ,   'Pz'  ,  'P4'  ,  'PO7'   , 'PO8'   , 'Oz']
    ch_types= ['eeg'] * signal.shape[1]

    ch_names_events = ch_names + ['t_stim']+ ['t_type']
    ch_types_events = ch_types + ['misc'] + ['misc']

    #info = mne.create_info(ch_names, 250, ch_types=ch_types)
    #eeg_mne = mne.io.array.RawArray(signal.T, info)

    signal_events = np.concatenate([signal, t_stim, t_type],1)
    info_events = mne.create_info(ch_names_events,250, ch_types_events)
    eeg = mne.io.RawArray(signal_events.T, info_events)

    # Do some basic signal processing (1-20 band pass filter)
    # fig=eeg.plot_psd()
    eeg.filter(1,20)
    # fig=eeg.plot_psd()
    event_times = mne.find_events(eeg, stim_channel='t_type')    
    # eeg.plot(scalings='auto',n_channels=8,events=event_times,block=True)   # scalings=10e-05

    if (np.unique(t_flash[:,0]).shape[0] != 4200):
        u,c = np.unique( t_flash[:,0], return_counts=True)
        dup = u[c>1]
        for i in range(dup.shape[0]):
            idx = np.where( t_flash[:,0] == dup[i] )[0][0]
            t_flash[idx,0]  -= 1
            t_flash[idx,1]  = 1
            t_type[t_flash[idx,0]] = t_flash[idx,3]
            t_stim[t_flash[idx,0]] = t_flash[idx,2]

    np.unique(t_flash[:,0]).shape
    assert  np.unique(t_flash[:,0]).shape[0] == 4200, 'Problem with experiment structure.  There aren''t enough events.'

    def getstims(eeg_mne, eeg_events):
        # Get the stimulations.  These are the FLASHINGS of rows and columns.
        tmin = 0
        tmax = 0.8
        reject = None
        event_times = mne.find_events(eeg_events, stim_channel='t_stim',shortest_event=0, verbose=True, min_duration=0.000001, consecutive=True)
        event_id = {'Row1':1,'Row2':2,'Row3':3,'Row4':4,'Row5':5,'Row6':6,'Col1':7,'Col2':8,'Col3':9,'Col4':10,'Col5':11,'Col6':12}

        epochs = mne.Epochs(eeg_mne, event_times, event_id, tmin, tmax, proj=False,
                        baseline=None, reject=reject, preload=True,
                        verbose=True, reject_by_annotation=None)

        stims = event_times[:,-1]
        return [epochs,stims]

    stimepochs, stims = getstims(eeg, eeg)

    def getlabels(eeg_mne, eeg_events, event_id):
        # Get the hit/no hits labels.  
        # These are the FLASHINGS of rows and columns but selected if they are the ones that will trigger the P300 response or not.
        # event_id = { 'first':1, 'second':2 }
        # baseline = (0.0, 0.2)
        # reject = {'eeg': 70 * pow(10,6)}
        tmin = 0
        tmax = 0.8
        reject = None
        event_times = mne.find_events(eeg_events, stim_channel='t_type', shortest_event=0, verbose=True, min_duration=0.000001, consecutive=True)
        epochs = mne.Epochs(eeg_mne, event_times, event_id, tmin, tmax, proj=False,
                        baseline=None, reject=reject, preload=True,
                        verbose=True, reject_by_annotation=None)
        labels = epochs.events[:, -1]
        return [epochs, labels]

    epochs, labels = getlabels(eeg, eeg, {'first':1})
    epocked = epochs.average()
    # epocked.plot(window_title='NoHit Averaged Signals')
    epochs, labels = getlabels(eeg, eeg, {'second':2})
    epocked = epochs.average()
    # epocked.plot(window_title='Hit Averaged Signals')
    epochs, labels = getlabels(eeg, eeg, { 'first':1, 'second':2})

    # Downsample the original FS=250 Hz signal to >>> 20 Hz
    #epochs.resample(20, npad="auto")
    #stimepochs.resample(20, npad="auto")
    # repetitions = 120
    # Con 100: El clasificador da resultados distintos.
    # %%
    # This is Single Flashing Classification attempt.
    from sklearn.preprocessing import  StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn import svm
    from sklearn.pipeline import make_pipeline

    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import classification_report, confusion_matrix
    from sklearn.preprocessing import MinMaxScaler

    # import a linear classifier from mne.decoding
    from mne.decoding import LinearModel

    import matplotlib.pyplot as plt

    clf = LogisticRegression(solver='lbfgs')
    scaler = StandardScaler()

    # create a linear model with LogisticRegression
    model = LinearModel(clf)

    # Get the epoched data (get only the data columns)
    eeg_data = epochs.get_data().reshape(len(labels), -1)
    eeg_data = eeg_data[:,0:epochs.get_data().shape[2]*1]
    #eeg_data[labels==2] = erptemplate1[:201,0]
    #eeg_data[labels==1] = erptemplate1[:201,0]
    #eeg_data[labels==2] = np.zeros((eeg_data.shape[1],))
    #eeg_data[labels==1] = np.ones((eeg_data.shape[1],))
    #labels = np.random.permutation(labels)

    # fit the classifier on MEG data
    X = scaler.fit_transform(eeg_data)

    model.fit(X[0:2800], labels[0:2800])

    preds = model.predict(X[2800:])

    # Classification report
    target_names = ['nohit', 'hit']

    report = classification_report(labels[2800:], preds, target_names=target_names)
    # print(report)

    cm = confusion_matrix(labels[2800:], preds)
    # print (cm)
    cm_normalized = cm.astype(float) / cm.sum(axis=1)[:, np.newaxis]
    acc=(cm[0,0]+cm[1,1])*1.0/(np.sum(cm))

    '''
    import matplotlib.pyplot as plt
    for i in range(200,220):
        plt.figure(figsize=(9, 3))
        plt.plot(eeg_data[labels==1][i])
        plt.show()
    '''

    # In[1]:   Average classification x trial (unbalanced)
    # Este dataset es un dataset de calibración. El registro de EEG corresponde a un experimento del Speller
    # de 35 letras (7 palabras de 5 letras).   Cada una de las letras consiste en 10 repeticiones de la intensificación 
    # de los 12 estímulos distintos, siendo cada estímulo el FLASHING de una de las 6 filas o 6 columnas.
    # Cada vez que se repite, se hace una permutación de los 12.
    # En cada una de esos 12 estímulos, dos, uno correspondiente a una fila y a una columna, corresponden 
    # a la letra que la persona está prestando atención y la idea es que el sistema descubra que letra es.

    # Primero tengo que agarrar la lista de labels y asignar a los 420 (35x12)
    # el label que le corresponde a cada uno.  Es decir de los primeros 12, 10
    # son no hits y 2 hits.

    # hlbls tiene pares (r,c) que representan la fila y la columna donde está la letra
    # que la persona tiene que elegir. 
    hlbls = []
    hpreds = []
    classlabels=np.asarray([])

    # ==================================================
    redondeo_hlbls = int((abs(min(len(stims), len(labels))))/120)
    print("hlbls:", redondeo_hlbls)
    # Este cálculo se hace porque no todos los resultados son iguales:
    # Paciente	            hlbls	Eventos
    # P300S01.mat	        33	    396
    # P300S02.mat	        33	    396
    # P300S03.mat	        32	    384
    # P300S04.mat	        32	    384
    # P300S05.mat	        33	    396
    # P300S06.mat	        32	    384
    # P300S07.mat	        32	    384
    # P300S08.mat	        32	    384
    #p300-subject-25.mat	35	    420
    # ==================================================

    for trial in range(0,redondeo_hlbls):
        a=np.zeros((12*10,2))
        a[:,0] = stims[0+120*trial:0+120*trial+120]
        a[:,1] = labels[0+120*trial:0+120*trial+120]    
        
        # b=np.zeros((12,1))
        b=np.zeros((12,2))
        
        for i in range(1,13):
            # print("valores únicos de a (stims):", np.unique(a[a[:,0]==i,1]))
            # print("a[:,0]:", a[:,0])
            # print("b[]:", b[i-1])
            b[i-1] = np.unique(a[a[:,0]==i,1])
            # b[i-1] = list(np.unique(a[a[:,0]==i,1]))

        b = b[:,1]
        
        for i in range(0,6):
            if (b[i]==2):
                r = i+1

        for i in range(6,12):
            if (b[i]==2):
                c = i+1
    
        classlabels = np.append( classlabels, b )
        assert (r!=0 and c!=0), 'Error %d,%d' % (r,c) 
        hlbls.append( (r,c) )
    print("hlbls:", len(hlbls))
    #print("len(classlabels):", len(classlabels))
    # print("classlabels:", classlabels)

    def SpellMeLetter(row, col):
        spellermatrix = [ ['A','B','C','D','E','F'],
                        [ 'G','H','I','J','K','L'],
                    [ 'M','N','O','P','Q','R'],
                    [ 'S','T','U','V','W','X'],
                    [ 'Y','Z','1','2','3','4'],
                    [ '5','6','7','8','9','_'] ]

        return spellermatrix[row-1][col-1-6]

    # print("Esta es la frase de 7 palabras de 5 letras que la persona tiene que producir.") 
    for i in range(0,redondeo_hlbls):
        print(SpellMeLetter(hlbls[i][0],hlbls[i][1]),end='')    
    print()

    # Luego necesito calcular los 420 averaging (de repetitions)
    # Finalmente aprendo con 180 y me fijo si predigo los 240
    # De los 240 adivino 20 letras (de a pares) y con eso calculo la performance

    def getaverageepoch(singleepoch):
        # Build the epochs based on each stimulation (1-12), and put all the epochs togheter.
        # Construye las épocas/eventos en función de cada estimulación (1-12) y junte todas las épocas/eventos.
        
        for trial in range(0,redondeo_hlbls):
            epochstrial = singleepoch[0+repetitions*trial:repetitions*trial+repetitions]
            '''
            print("epochr1:", epochr1)
            print("epochr1.shape:", epochr1.data().shape)
            print("epochr1:", epochr1.get_data())
            print("epochr1.get_data.shape:", epochr1.get_data().shape)
            print("epochr1.average.data.shape:", [epochr1.average().data.shape])
            #print("epochs_data", epochs_data)
            '''
            epochr1 = epochstrial['Row1'] # print("epochr1:", epochr1) # print("epochstrial['Row1']:", epochstrial['Row1'])
            epochr2 = epochstrial['Row2'] # print("epochstrial['Row2']:", epochstrial['Row2'])
            epochr3 = epochstrial['Row3']
            epochr4 = epochstrial['Row4']
            epochr5 = epochstrial['Row5']
            epochr6 = epochstrial['Row6']
            
            epochc1 = epochstrial['Col1']
            epochc2 = epochstrial['Col2']
            epochc3 = epochstrial['Col3']
            epochc4 = epochstrial['Col4']
            epochc5 = epochstrial['Col5']
            epochc6 = epochstrial['Col6']
            
            if (trial==0):
                epochs_data = np.array([epochr1.average().data])
                #print("epochs_data.shape:", epochs_data.shape)
            else:
                epochs_data = np.concatenate((epochs_data, [epochr1.average().data]), axis=0)
                #print("epochr1.average().data.shape:", epochr1.average().data.shape)
            epochs_data = np.concatenate((epochs_data, [epochr2.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochr3.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochr4.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochr5.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochr6.average().data]), axis=0)

            epochs_data = np.concatenate((epochs_data, [epochc1.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochc2.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochc3.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochc4.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochc5.average().data]), axis=0)
            epochs_data = np.concatenate((epochs_data, [epochc6.average().data]), axis=0)

            # print(f'epochs_data: ', epochs_data)

        # There are 420 epochs, which correspond to 35 letters, in groups of 12.
        events=np.array([np.arange(len(classlabels)),np.zeros(len(classlabels)), classlabels])
        events = events.T
        events = events.astype(int)
                        
        tmin = 0
        tmax = 0.8
        event_id = { 'first':1, 'second':2 }
        info = mne.create_info(ch_names, 250, ch_types=ch_types)
        custom_epochs = mne.EpochsArray(epochs_data, info, events, tmin, event_id) 

        return custom_epochs

    # avepochs contains all the 420 averaged epochs, 7 letters of 5, with 12 each.
    custom_epochs = getaverageepoch(stimepochs)          

    # Performs the final classification, the one that allows to produce the spelled letters.
    print('Performance Classification of Averaged Epochs')
    clf = LogisticRegression(solver='lbfgs')
    scaler = StandardScaler()

    # create a linear model with LogisticRegression
    model = LinearModel(clf)

    # =============================================
    # Acá redimensiono el dataset para que training y test queden 
    # en función de la cantidad de eventos reales encontrados.
    # no 420 exactos.
    eventos_encontrados = len(custom_epochs)  
    long_training = int(abs(len(custom_epochs))*(0.43))
    long_test = ((len(custom_epochs))-long_training)
    #redondeo_hlbls
    redondeo_hlbls_40 = int(abs(redondeo_hlbls*(0.4)))
    #print("redondeo_hlbls_40:", redondeo_hlbls_40)
    # =============================================

    # print("len(eventos_encontrados):", len(custom_epochs))
    # print("long_training:", long_training)
    # print("long_test:", long_test)
    # print("redondeo_hlbls:", redondeo_hlbls)
    # print("redondeo_hlbls_40:", redondeo_hlbls_40)

    # training = range(0,180)
    # test = range(180,420)

    training = range(0,long_training)
    test = range(long_training,eventos_encontrados)

    #print("training:", training)
    #print("test:", test)

    eeg_data = custom_epochs.get_data()

    #print("Tipo (eeg_data) : ", type(eeg_data))
    #print("len(eeg_data):", len(eeg_data))
    # print("shape(eeg_data):", eeg_data.shape)
    #print("eeg_data:", eeg_data)
    #np.savetxt(f'./a_results/eeg_data.csv', eeg_data, delimiter=',',fmt='%s')
    # np.savetxt(f'./a_results/events2.csv', events, delimiter='\t',fmt='%s')

    eeg_data = eeg_data.reshape(eventos_encontrados, -1)

    # print("re-shape(eeg_data):", eeg_data.shape)

    eeg_data = eeg_data[:,0:201]

    #eeg_data[classlabels==2] = np.zeros((eeg_data.shape[1],))
    #eeg_data[classlabels==1] = np.ones((eeg_data.shape[1],))

    #X = scaler.fit_transform(eeg_data)

    X = eeg_data

    cf = clf.fit(X[training], classlabels[training])

    classpreds = np.empty ((eventos_encontrados,2))

    classpreds[test,:] = clf.predict_proba(X[test])

    hpreds = []

    for trial in range(redondeo_hlbls_40,redondeo_hlbls):
    #for trial in range(15,35):
        #print('Row')
        for i in range(0,6):
            preds = classpreds[trial*12+i]
            #print ( preds[1] )
            labels = classlabels[trial*12+i]

        #print (  np.argmin( classpreds[trial*12+0:trial*12+6]))
        r = np.argmax( classpreds[trial*12+0:trial*12+6,1])+1
        
        #print('Col')
        for i in range(6,12):
            preds = classpreds[trial*12+i]
            #print ( preds[1] )
            labels = classlabels[trial*12+i]

        #print (  np.argmin( classpreds[trial*12+6:trial*12+12]))
        c = np.argmax( classpreds[trial*12+6:trial*12+12,1])+1

        hpreds.append( (r,c) )

    # print("lo esperado:")
    for i in range(redondeo_hlbls_40,redondeo_hlbls):
    #for i in range(15,35):
        print(SpellMeLetter(hlbls[i][0],hlbls[i][1]),end='')
    print()

    # print("lo predecido")
    for i in range(redondeo_hlbls_40,redondeo_hlbls):
    #for i in range(15,35):
        print(SpellMeLetter(hpreds[i-15][0],hpreds[i-15][1]),end='')
    print()

    '''
    # %%
    for i in range(0,12):
        plt.figure(figsize=(9, 3))
        plt.plot(eeg_data[i])
        plt.show()
    # %%
    '''

    # Classification report
    target_names = ['nohit', 'hit']
    report = classification_report(classlabels[test], clf.predict(X[test]), target_names=target_names)
    # print(report)
    # print(f'report: ', report)
    cm = confusion_matrix(classlabels[test], clf.predict(X[test]) )
    # print (cm)
    cm_normalized = cm.astype(float) / cm.sum(axis=1)[:, np.newaxis]
    acc=(cm[0,0]+cm[1,1])*1.0/(np.sum(cm))
    # print("cm:", cm)
    print("-----------------------------------------")  
    print(f'Accuracy P3S00{ii+1}: {acc}') # COLUMNA0.
    print(f'P{ii+1}, {acc-0.2}, {repetitions}', file=file_temp)
    print("-----------------------------------------")  

file_temp.close()


vector_desfase:  [1, 2, 3, 4, 5]
4040 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12]
Not setting metadata
4040 matching events found
No baseline correction applied
Using data from preloaded Raw for 4040 events and 201 original time points ...
0 bad epochs dropped
4004 events found
Event IDs: [1 2]
Not setting metadata
3333 matching events found
No baseline correction applied
Using data from preloaded Raw for 3333 events and 201 original time points ...
0 bad epochs dropped
4004 events found
Event IDs: [1 2]
Not setting metadata
671 matching events found
No baseline correction applied
Using data from preloaded Raw for 671 events and 201 original time points ...
0 bad epochs dropped
4004 events found
Event IDs: [1 2]
Not setting metadata
4004 matching events found
No baseline correction applied
Using data from preloaded Raw for 4004 events and 201 original time points ...
0 bad epochs dropped


c:\Users\alexc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


hlbls: 33
hlbls: 33
9_944___9_____9_________4___9_4_2
Performance Classification of Averaged Epochs
_9_________4___9_4_2
OYO_8FHENF2H23SK9LPK
-----------------------------------------
Accuracy P3S001: 0.7522123893805309
-----------------------------------------


c:\Users\alexc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


4065 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12]
Not setting metadata
4065 matching events found
No baseline correction applied
Using data from preloaded Raw for 4065 events and 201 original time points ...
2 bad epochs dropped
4018 events found
Event IDs: [1 2]
Not setting metadata
3337 matching events found
No baseline correction applied
Using data from preloaded Raw for 3337 events and 201 original time points ...
2 bad epochs dropped
4018 events found
Event IDs: [1 2]
Not setting metadata
681 matching events found
No baseline correction applied
Using data from preloaded Raw for 681 events and 201 original time points ...
0 bad epochs dropped
4018 events found
Event IDs: [1 2]
Not setting metadata
4018 matching events found
No baseline correction applied
Using data from preloaded Raw for 4018 events and 201 original time points ...
2 bad epochs dropped
hlbls: 33
hlbls: 33
__4_44948__4_____9___9___________


...

c:\Users\alexc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


3985 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12]
Not setting metadata
3985 matching events found
No baseline correction applied
Using data from preloaded Raw for 3985 events and 201 original time points ...
2 bad epochs dropped
3896 events found
Event IDs: [1 2]
Not setting metadata
3246 matching events found
No baseline correction applied
Using data from preloaded Raw for 3246 events and 201 original time points ...
2 bad epochs dropped
3896 events found
Event IDs: [1 2]
Not setting metadata
650 matching events found
No baseline correction applied
Using data from preloaded Raw for 650 events and 201 original time points ...
0 bad epochs dropped
3896 events found
Event IDs: [1 2]
Not setting metadata
3896 matching events found
No baseline correction applied
Using data from preloaded Raw for 3896 events and 201 original time points ...
2 bad epochs dropped
hlbls: 32
hlbls: 32
_9____4____9___9___________9_9__


c:\Users\alexc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Performance Classification of Averaged Epochs
___9___________9_9__
YAH_9P5KVY4OS6XWMQ19
-----------------------------------------
Accuracy P3S004: 0.7808219178082192
-----------------------------------------


c:\Users\alexc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
